In [64]:
# coding=UTF-8
import jieba
import json
import codecs

import sys
import os
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn import feature_extraction 
 
import numpy as np
import scipy

import lda  
import lda.datasets 
jieba.set_dictionary('dict.txt.big')
jieba.load_userdict('繁體中文詞庫.txt')

f = open('stop_words.txt', 'r',encoding="utf-8")
stopwords = []  
for line in f.readlines():
    stopwords.append(line.strip()) 
f.close() 

data_name = ['3c_dcard', 'acg_dcard', 'boy_dcard', 'buyonline_dcard',
             'food_dcard', 'game_dcard', 'girl_dcard', 'makeup_dcard',
             'money_dcard', 'movie_dcard', 'sport_dcard', 'talk_dcard',
             'travel_dcard', 'tvepisode_dcard', 'vehicle_dcard']


    
for j in range(len(data_name)):
    file = open("news_materials_Dcard\\"+data_name[j]+".json", 'r', encoding='utf-8')
    f = file.read()
    jsondata = json.loads(f)
    file.close()
    
    content_cut = []
    content_title = []
    content_excerpt = []
    
    for content in jsondata:
        content_str = ''
        word_str = ''
        train = []
        content_title.append(content['title']) 
        content_excerpt.append(content['excerpt'])
        line = jieba.cut(content['title']+content['excerpt'])
        train.append([w for w in line if w not in stopwords])
        
        
        for word_1 in train[0]:
            if word_1 == ' ':
                continue
            word_str = word_str+' ' + word_1
        content_cut.append(word_str)

#將文字傳化成陣列
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(content_cut)
    analyze = vectorizer.build_analyzer()
    weight = X.toarray()  


    print (len(weight))  
    print (weight[:5,:5])
 
    model = lda.LDA(n_topics=4, n_iter=500, random_state=1)  
    model.fit(np.asarray(weight))     # model.fit_transform(X) is also available  
    topic_word = model.topic_word_ 

#文档-主题（Document-Topic）分布 
    
    doc_topic = model.doc_topic_  
    #print("type(doc_topic): {}".format(type(doc_topic)))  
    #print("shape: {}".format(doc_topic.shape))
    #输出前10篇文章最可能的Topic  
    label = [] 
    topic0_count = 0
    topic1_count = 0
    topic2_count = 0
    topic3_count = 0
    for n in range(len(weight)):  
        topic_most_pr = doc_topic[n].argmax()  
        label.append(topic_most_pr)  
        if topic_most_pr == 0:
            topic0_count += 1
        elif topic_most_pr == 1:
            topic1_count += 1
        elif topic_most_pr == 2:
            topic2_count += 1
        elif topic_most_pr == 3:
            topic3_count += 1
        #print("doc: {} topic: {}".format(n, topic_most_pr))
    
    
#输出主题中的TopN关键词  
    word = vectorizer.get_feature_names()  

    #print (topic_word[:, :5])  
    n = 5   
    for i, topic_dist in enumerate(topic_word):    
        topic_words = np.array(word)[np.argsort(topic_dist)][:-(n+1):-1]    
        #print(u'*Topic {}\n- {}'.format(i, ' '.join(topic_words)))
        if i == 0:
            topic0 =  ' '.join(topic_words)
        elif i == 1:
            topic1 =  ' '.join(topic_words)
        elif i == 2:
            topic2 =  ' '.join(topic_words)
        elif i == 3:
            topic3 =  ' '.join(topic_words)
    
    all_list = []
    for t in range(len(label)):
        content_dict = {
            'topic' : int(label[t]),
            'title' : content_title[t],
            'excerpt' :content_excerpt[t]
        } 
        
        all_list.append(content_dict)
   
    topic_dict = {
        'topic0':topic0,
        'topic1':topic1,
        'topic2':topic2,
        'topic3':topic3,
        'topic0_count':topic0_count,
        'topic1_count':topic1_count,
        'topic2_count':topic2_count,
        'topic3_count':topic3_count
    }
    all_dict = {
        'all_content' : all_list,
        'topic_dict' : topic_dict
    }
    
    with codecs.open('news_materials_Dcard_分析\\'+data_name[j]+'_分析.json', 'w', 'utf-8') as file2:
        json.dump(all_dict, file2,indent = 5,sort_keys=True,ensure_ascii=False)
    file2.close()

Building prefix dict from C:\Users\user\dict.txt.big ...
DEBUG:jieba:Building prefix dict from C:\Users\user\dict.txt.big ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.ua7677c818aa16337f448e65eab47ca3f.cache
DEBUG:jieba:Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.ua7677c818aa16337f448e65eab47ca3f.cache
Loading model cost 1.961 seconds.
DEBUG:jieba:Loading model cost 1.961 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.
INFO:lda:n_documents: 100
INFO:lda:vocab_size: 1147
INFO:lda:n_words: 2088
INFO:lda:n_topics: 4
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -22212
INFO:lda:<10> log likelihood: -18029
INFO:lda:<20> log likelihood: -17860


100
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


INFO:lda:<30> log likelihood: -17689
INFO:lda:<40> log likelihood: -17578
INFO:lda:<50> log likelihood: -17590
INFO:lda:<60> log likelihood: -17444
INFO:lda:<70> log likelihood: -17454
INFO:lda:<80> log likelihood: -17448
INFO:lda:<90> log likelihood: -17377
INFO:lda:<100> log likelihood: -17383
INFO:lda:<110> log likelihood: -17415
INFO:lda:<120> log likelihood: -17340
INFO:lda:<130> log likelihood: -17333
INFO:lda:<140> log likelihood: -17297
INFO:lda:<150> log likelihood: -17280
INFO:lda:<160> log likelihood: -17398
INFO:lda:<170> log likelihood: -17339
INFO:lda:<180> log likelihood: -17295
INFO:lda:<190> log likelihood: -17358
INFO:lda:<200> log likelihood: -17234
INFO:lda:<210> log likelihood: -17254
INFO:lda:<220> log likelihood: -17294
INFO:lda:<230> log likelihood: -17330
INFO:lda:<240> log likelihood: -17278
INFO:lda:<250> log likelihood: -17364
INFO:lda:<260> log likelihood: -17370
INFO:lda:<270> log likelihood: -17331
INFO:lda:<280> log likelihood: -17306
INFO:lda:<290> log 

100
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]]


INFO:lda:<20> log likelihood: -17225
INFO:lda:<30> log likelihood: -17059
INFO:lda:<40> log likelihood: -17056
INFO:lda:<50> log likelihood: -16961
INFO:lda:<60> log likelihood: -16891
INFO:lda:<70> log likelihood: -16941
INFO:lda:<80> log likelihood: -16904
INFO:lda:<90> log likelihood: -16800
INFO:lda:<100> log likelihood: -16767
INFO:lda:<110> log likelihood: -16753
INFO:lda:<120> log likelihood: -16877
INFO:lda:<130> log likelihood: -16821
INFO:lda:<140> log likelihood: -16839
INFO:lda:<150> log likelihood: -16768
INFO:lda:<160> log likelihood: -16833
INFO:lda:<170> log likelihood: -16822
INFO:lda:<180> log likelihood: -16869
INFO:lda:<190> log likelihood: -16769
INFO:lda:<200> log likelihood: -16712
INFO:lda:<210> log likelihood: -16787
INFO:lda:<220> log likelihood: -16786
INFO:lda:<230> log likelihood: -16807
INFO:lda:<240> log likelihood: -16822
INFO:lda:<250> log likelihood: -16740
INFO:lda:<260> log likelihood: -16841
INFO:lda:<270> log likelihood: -16808
INFO:lda:<280> log l

...

INFO:lda:<50> log likelihood: -21919
INFO:lda:<60> log likelihood: -21775
INFO:lda:<70> log likelihood: -21708
INFO:lda:<80> log likelihood: -21786
INFO:lda:<90> log likelihood: -21691
INFO:lda:<100> log likelihood: -21668
INFO:lda:<110> log likelihood: -21636
INFO:lda:<120> log likelihood: -21585
INFO:lda:<130> log likelihood: -21669
INFO:lda:<140> log likelihood: -21578
INFO:lda:<150> log likelihood: -21572
INFO:lda:<160> log likelihood: -21572
INFO:lda:<170> log likelihood: -21570
INFO:lda:<180> log likelihood: -21658
INFO:lda:<190> log likelihood: -21675
INFO:lda:<200> log likelihood: -21642
INFO:lda:<210> log likelihood: -21531
INFO:lda:<220> log likelihood: -21597
INFO:lda:<230> log likelihood: -21551
INFO:lda:<240> log likelihood: -21496
INFO:lda:<250> log likelihood: -21546
INFO:lda:<260> log likelihood: -21541
INFO:lda:<270> log likelihood: -21587
INFO:lda:<280> log likelihood: -21615
INFO:lda:<290> log likelihood: -21565
INFO:lda:<300> log likelihood: -21624
INFO:lda:<310> lo

100
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


INFO:lda:<40> log likelihood: -20415
INFO:lda:<50> log likelihood: -20393
INFO:lda:<60> log likelihood: -20385
INFO:lda:<70> log likelihood: -20395
INFO:lda:<80> log likelihood: -20329
INFO:lda:<90> log likelihood: -20312
INFO:lda:<100> log likelihood: -20227
INFO:lda:<110> log likelihood: -20306
INFO:lda:<120> log likelihood: -20329
INFO:lda:<130> log likelihood: -20289
INFO:lda:<140> log likelihood: -20349
INFO:lda:<150> log likelihood: -20257
INFO:lda:<160> log likelihood: -20222
INFO:lda:<170> log likelihood: -20260
INFO:lda:<180> log likelihood: -20281
INFO:lda:<190> log likelihood: -20303
INFO:lda:<200> log likelihood: -20220
INFO:lda:<210> log likelihood: -20178
INFO:lda:<220> log likelihood: -20238
INFO:lda:<230> log likelihood: -20276
INFO:lda:<240> log likelihood: -20228
INFO:lda:<250> log likelihood: -20204
INFO:lda:<260> log likelihood: -20234
INFO:lda:<270> log likelihood: -20185
INFO:lda:<280> log likelihood: -20168
INFO:lda:<290> log likelihood: -20165
INFO:lda:<300> log

100
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


INFO:lda:<50> log likelihood: -18264
INFO:lda:<60> log likelihood: -18273
INFO:lda:<70> log likelihood: -18154
INFO:lda:<80> log likelihood: -18186
INFO:lda:<90> log likelihood: -18120
INFO:lda:<100> log likelihood: -18142
INFO:lda:<110> log likelihood: -18131
INFO:lda:<120> log likelihood: -18162
INFO:lda:<130> log likelihood: -18132
INFO:lda:<140> log likelihood: -18146
INFO:lda:<150> log likelihood: -18066
INFO:lda:<160> log likelihood: -18038
INFO:lda:<170> log likelihood: -18077
INFO:lda:<180> log likelihood: -18010
INFO:lda:<190> log likelihood: -18060
INFO:lda:<200> log likelihood: -18065
INFO:lda:<210> log likelihood: -18009
INFO:lda:<220> log likelihood: -18089
INFO:lda:<230> log likelihood: -18094
INFO:lda:<240> log likelihood: -18049
INFO:lda:<250> log likelihood: -18051
INFO:lda:<260> log likelihood: -18058
INFO:lda:<270> log likelihood: -18043
INFO:lda:<280> log likelihood: -18072
INFO:lda:<290> log likelihood: -18078
INFO:lda:<300> log likelihood: -18113
INFO:lda:<310> lo

In [44]:
data_name = []

for filename in os.listdir('news_materials_Dcard'):
    data_name.append(filename)
print(data_name)

['3c_dcard.json', 'acg_dcard.json', 'boy_dcard.json', 'buyonline_dcard.json', 'food_dcard.json', 'game_dcard.json', 'girl_dcard.json', 'makeup_dcard.json', 'money_dcard.json', 'movie_dcard.json', 'sport_dcard.json', 'talk_dcard.json', 'travel_dcard.json', 'tvepisode_dcard.json', 'vehicle_dcard.json']
